In [16]:
import tensorflow as tf
import pandas as pd
from sklearn import model_selection

In [127]:
cols = ["nlikes", "nreplies", "nretweets", "close", "date"]
data = pd.read_csv('data/needed/cleaned/Tweets_and_Stock.csv', usecols=cols)
data = data.dropna(subset=["nlikes", "close"])
print(data.shape)

(10622, 5)


## Model rounded to next 100

In [84]:
#target use close price rounded to next 100
data["close"] = data["close"].apply(lambda x: int(round(x/100)*100))
data["close"] = data["close"].factorize()[0]
data.describe()

,nlikes,nreplies,nretweets,close
count,1.062200e+04,10622.000000,10622.000000,10622.000000
mean,2.449201e+04,1093.618810,2467.577198,8.955846
std,7.101487e+04,4041.571601,10172.791564,3.588659
min,2.000000e+00,0.000000,0.000000,0.000000
25%,1.129250e+03,57.000000,59.000000,6.000000
50%,3.747500e+03,171.000000,219.000000,11.000000
75%,1.403675e+04,599.000000,989.750000,11.000000
max,1.596525e+06,120523.000000,362906.000000,12.000000


In [85]:
train, test = model_selection.train_test_split(data, test_size= 0.2)
print(train.shape)
print(test.shape)

train_in = train[["nlikes", "nreplies", "nretweets"]]
train_target = train["close"]

test_in = test[["nlikes", "nreplies", "nretweets"]]
test_target = test["close"]

(8497, 4)
(2125, 4)


In [98]:
normalizer = tf.keras.layers.Normalization(axis=-1) #to make values between 0 and 1
normalizer.adapt(train_in)

model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(max(test_target)+1)
  ])

model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [99]:
model.fit(train_in, train_target, epochs=10, batch_size=10)

Epoch 1/10
850/850 [==============================] - 1s 809us/step - loss: 2.1746 - accuracy: 0.3670
Epoch 2/10
850/850 [==============================] - 1s 843us/step - loss: 1.9520 - accuracy: 0.3834
Epoch 3/10
850/850 [==============================] - 1s 845us/step - loss: 1.9136 - accuracy: 0.3848
Epoch 4/10
850/850 [==============================] - 1s 795us/step - loss: 1.8920 - accuracy: 0.3865
Epoch 5/10
850/850 [==============================] - 1s 887us/step - loss: 1.8687 - accuracy: 0.3874
Epoch 6/10
850/850 [==============================] - 1s 795us/step - loss: 1.8413 - accuracy: 0.3879
Epoch 7/10
850/850 [==============================] - 1s 796us/step - loss: 1.8221 - accuracy: 0.3873
Epoch 8/10
850/850 [==============================] - 1s 809us/step - loss: 1.8003 - accuracy: 0.3894
Epoch 9/10
850/850 [==============================] - 1s 802us/step - loss: 1.7864 - accuracy: 0.3897
Epoch 10/10
850/850 [==============================] - 1s 832us/step - loss: 1.773

In [100]:
model.evaluate(test_in, test_target)

67/67 [==============================] - 0s 773us/step - loss: 1.7146 - accuracy: 0.3948


[1.7145901918411255, 0.39482352137565613]

## Model rounded to next 1000 --> Binary

In [74]:
data["close"] = data["close"].apply(lambda x: int(round(x/1000)*1000))
data["close"] = data["close"].factorize()[0]

train, test = model_selection.train_test_split(data, test_size= 0.2)

train_in = train[["nlikes", "nreplies", "nretweets"]]
train_target = train["close"]

test_in = test[["nlikes", "nreplies", "nretweets"]]
test_target = test["close"]

In [80]:
normalizer = tf.keras.layers.Normalization(axis=-1) #to make values between 0 and 1
normalizer.adapt(train_in)

model_bin = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1)
  ])

model_bin.compile(optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [81]:
model_bin.fit(train_in, train_target, epochs=10, batch_size=50)

Epoch 1/10
170/170 [==============================] - 1s 898us/step - loss: 0.6124 - accuracy: 0.4698
Epoch 2/10
170/170 [==============================] - 0s 940us/step - loss: 0.5484 - accuracy: 0.7419
Epoch 3/10
170/170 [==============================] - 0s 939us/step - loss: 0.5307 - accuracy: 0.7533
Epoch 4/10
170/170 [==============================] - 0s 1ms/step - loss: 0.5142 - accuracy: 0.7670
Epoch 5/10
170/170 [==============================] - 0s 931us/step - loss: 0.4907 - accuracy: 0.7852
Epoch 6/10
170/170 [==============================] - 0s 886us/step - loss: 0.4725 - accuracy: 0.7939
Epoch 7/10
170/170 [==============================] - 0s 869us/step - loss: 0.4527 - accuracy: 0.8065
Epoch 8/10
170/170 [==============================] - 0s 891us/step - loss: 0.4399 - accuracy: 0.8125
Epoch 9/10
170/170 [==============================] - 0s 856us/step - loss: 0.4217 - accuracy: 0.8223
Epoch 10/10
170/170 [==============================] - 0s 889us/step - loss: 0.4156 

In [82]:
model_bin.evaluate(test_in, test_target)

67/67 [==============================] - 0s 696us/step - loss: 0.3829 - accuracy: 0.8273


[0.3828790485858917, 0.827294111251831]

## Binary up or down per day

In [128]:
data = data.groupby(["date"])

data1 = data.sum() # for tweet statistics
data2 = data.mean() # for stock price
data1["close"] = data2["close"]
data = data1

# function to assign 1 if price went up, 0 otherwise
def direction (x):
    if x > 0:
        return 1
    else:
        return 0

# assign 1 or 0 to every date
data["close_diff"] = data["close"].diff()
data["direction"] = data["close_diff"].apply(direction)
print(data)

train, test = model_selection.train_test_split(data, test_size= 0.2)

train_in = train[["nlikes", "nreplies", "nretweets"]]
train_target = train["direction"]

test_in = test[["nlikes", "nreplies", "nretweets"]]
test_target = test["direction"]

              nlikes  nreplies  nretweets    close  close_diff  direction
date                                                                     
2011-12-01     267.0      63.0       24.0    6.520         NaN          0
2011-12-21    1330.0      87.0      597.0    5.514      -1.006          0
2011-12-22    1349.0     132.0      206.0    5.554       0.040          1
2011-12-27      39.0      13.0       34.0    5.714       0.160          1
2011-12-28      12.0       7.0        1.0    5.702      -0.012          0
...              ...       ...        ...      ...         ...        ...
2022-02-25  118902.0    9995.0     6482.0  809.870      45.830          1
2022-03-01  259382.0    7631.0    14625.0  864.370      54.500          1
2022-03-02  496142.0   19800.0    51087.0  879.890      15.520          1
2022-03-03  688520.0   21765.0    73639.0  839.290     -40.600          0
2022-03-04   95202.0    4688.0     4779.0  838.290      -1.000          0

[1636 rows x 6 columns]


In [129]:
normalizer = tf.keras.layers.Normalization(axis=-1) #to make values between 0 and 1
normalizer.adapt(train_in)

model_bin = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1)
  ])

model_bin.compile(optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [130]:
model_bin.fit(train_in, train_target, epochs=10, batch_size=5)

Epoch 1/10
262/262 [==============================] - 1s 855us/step - loss: 0.6981 - accuracy: 0.4572
Epoch 2/10
262/262 [==============================] - 0s 839us/step - loss: 0.6934 - accuracy: 0.4648
Epoch 3/10
262/262 [==============================] - 0s 837us/step - loss: 0.6935 - accuracy: 0.4625
Epoch 4/10
262/262 [==============================] - 0s 868us/step - loss: 0.6945 - accuracy: 0.4595
Epoch 5/10
262/262 [==============================] - 0s 946us/step - loss: 0.6941 - accuracy: 0.4511
Epoch 6/10
262/262 [==============================] - 0s 958us/step - loss: 0.6916 - accuracy: 0.4602
Epoch 7/10
262/262 [==============================] - 0s 940us/step - loss: 0.6952 - accuracy: 0.4557
Epoch 8/10
262/262 [==============================] - 0s 852us/step - loss: 0.6888 - accuracy: 0.4595
Epoch 9/10
262/262 [==============================] - 0s 849us/step - loss: 0.6924 - accuracy: 0.4580
Epoch 10/10
262/262 [==============================] - 0s 840us/step - loss: 0.689

In [131]:
model_bin.evaluate(test_in, test_target)

11/11 [==============================] - 0s 886us/step - loss: 0.6944 - accuracy: 0.4878


[0.694447934627533, 0.4878048896789551]